### Opdracht 
We gaan proberen om een voorspelling te doen over de uitslag van een (verkleinde) voetbalpool om te zien wie er waarschijnlijk gaat winnen. We gaan deze voorspelling doen aan de hand van de  Random Number Generator en het principe vanMonte Carlo Simulatie.

Maak een Monte Carlo Simulatie die meerdere keren (duizenden) de competitie speelt met gebruik van deze tabel van kansen. Hoe vaker je een competitie speelt, hoe nauwkeuriger je voorspelling wordt. Houd bij hoeveel punten elk team scoort in de competitie (3 punten voor winst, 1 punt voor gelijk, 0 voor verlies), en maak een overzicht (ranking) van de teams.

<img src = "tabelMC.PNG">

In [ ]:
import random
import pandas as pd
import numpy as np
import math 

Zet bovenstaande tabel om in een dictionary

In [ ]:
clubs = {
    "ajax":{"feyenoord":[65,17,18], "psv":[54,21,25], "fcutrecht":[74,14,12], "willemII":[78,13,9]},
    "feyenoord": {"ajax": [30,21,49], "psv":[37,24,39], "fcutrecht":[51,22,27], "willemII":[60,21,19]},
    "psv": {"ajax": [39,22,39], "feyenoord":[54,22,24], "fcutrecht":[62,20,18], "willemII": [62,22,16]},
    "fcutrecht": {"ajax":[25,14,61], "feyenoord": [37,23,40], "psv":[29,24,47], "willemII": [53,23,25]},
    "willemII": {"ajax": [17, 18, 65], "feyenoord":[20,26,54], "psv":[23,24,53], "fcutrecht":[37,25,38]}
}

Een functie maken waar we willekeurige getallen genereren op basis van de Linear congruential method.
De formule voor de Linear Congruential method luid als volgt: <br>
**X n+1 = (a * Xn + c) mod m**
waar
* a= multiplier
* c = increment
* Xn als n = 0 => seed (starting value)
* m = modulus


In [ ]:
class LCR:
    
    def __init__(self, a, c, m, seed):
        self.a = a
        self.c = c
        self.m = m
        self.seed = seed
        
    def random_number_generator(self):
        "Generating a random number using linear congruential method "
    #     x=[seed] # list containing n random generated numbers 
        x_next = (self.a * self.seed + self.c) % self.m
        self.seed = x_next 
        return x_next % 100
    


In [ ]:
rn = LCR(1664525, 1013904223, 2 ** 32,  20200420162000)
lst = []
for i in range(100):
    lst.append(rn.random_number_generator())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(lst)

In [ ]:
plt.hist(lst)

Definieer een spel tussen 2 teams. <br>
De scores luidt als volgt:
* 3 punten voor winst
* 1 punt voor gelijk spel
* 0 punten voor verlies

In [ ]:
import random 

In [ ]:
def match(team_lst, rn):
    
    #vb ayax - feyenoord [65, 17, 18]
    if  rn < team_lst[2]: # rn < 18 , dan verlies thuisspelende partij
        return 0, 3
    elif rn < (team_lst[2] + team_lst [1]): # rn zit tussen 18 en  35 (18+17), dan is er een gelijkspel
        return 1,1
    else: # rn zit tussen 35 en 100 (35 + 65), dan win de thuisspelende partij
        return 3, 0
    

In [ ]:
# Implement tournament
# A tournament is every team plays against each other once away and once at home.
def tournament(clubs, rn):
    #initialise standing dict to keep track of the points in the tournament
    standing = {}
    for team in clubs.keys():
        standing[team] = 0
        
    for home_team in clubs.keys(): # for each team , each team will play as hometeam
        for away_team in clubs[home_team].keys():# get the list from the home_team against away_team
            random_number = rn.random_number_generator()  #  pakt random getal
            probablity_list = clubs[home_team][away_team] # pak de lijst van kansen voor de away team van de ene home team
            ht_point, at_point = match(probablity_list, random_number) # pakt de punten 
            
            standing[home_team]+= ht_point # voeg aantal punten toe voor thuisspelende partij
            standing[away_team]+= at_point # voeg aantal punten toe voor uitspelende partij

    return standing
            

In [ ]:
s = tournament(clubs, rn)
s

In [ ]:
def rankings(standings):
    rank_comp   = {k: v for k, v in sorted(standings.items(), key=lambda item: item[1], reverse= True)}# dictionary sorteren
    return rank_comp.keys()

In [ ]:
rankings(s)

Definieer competities waarbij een team thuis speelt met de rest van de teams (uitploegen).

Maak Monte carlo functie die de functie championships n keren draait en de resultaten opslaan in average_ranks

In [ ]:
def MC(clubs, n):
    # generate dict of average ranking of the teams
    average_ranks = {}
    
    rn = LCR(1664525, 1013904223, 2 ** 32,  20200420162000)
    for team in clubs.keys():
        average_ranks[team] = {}
        for pos in range(1, 6):
            average_ranks[team][pos] = 0
    
    for i in range(0, n):
        standings = tournament(clubs, rn)
        sorted_standings = rankings(standings)
#         print(standings)
        
        for pos in range(len(sorted_standings)):
            team = list(sorted_standings)[pos]
            average_ranks[team][pos+1] +=1
            
        
    return average_ranks

In [ ]:
n = 1000
res = MC(clubs,n)
res

 Zet dictionaries om in een dataframe 

In [ ]:
df = pd.DataFrame(res)
def calculate_probability(x):
    return (x / n )*100 
df2 = df.apply(np.vectorize(calculate_probability))
print("Amount of competition that each team has had to get to that place.")
display(df)
print("Probability of each team getting to that place in a competition.")
display(df2)

Elke wedstrijd heeft een plaats in het eindklassement (rangschikking 1e, 2e, 3e, 4e en 5e). De waarschijnlijkheid dat een team een ​​specifieke plaats behaalt in de laatste klassering = het aantal keren dat het team op een specifieke plaats is geclassificeerd / totaal aantal competities in een kampioenschap.

In onze Monte carlo-functie voeren we n kampioenschappen uit. <br>
**x** = het aantal keren dat het team op een specifieke plaats is geclassificeerd<br>
**n** = het aantal iteratie van je MC<br>
De kans zal zijn **x/n**


## Verdiepende opdracht

In [ ]:
class Team:
  
    def __init__(self, average_home_score, average_home_conceded,average_away_score ,average_away_conceded):
        self.ahs = average_home_score
        self.ahc = average_home_conceded
        self.aas = average_away_score
        self.aac = average_away_conceded
        
    def aanvalskracht(self): # 'Attack Strength'
            pass
    
    def verdedigingskracht(self): # 'Defense Strength'
        pass
    
        
        
        

In [ ]:
class Tournament:
    
    def __init__(self):
        self.teams = []
    
    def gemiddelde_thuis_scoort(self):
        sum_ahs = 0
        for team in self.teams:
            sum_ahs += team.ahs
        
        return sum_ahs/len(self.teams)
    
    def gemiddelde_thuis_toegegeven(self):
        sum_ahc = 0
        for team in self.teams:
            sum_ahc += team.ahc
        
        return sum_ahc/len(self.teams)
    
    def gemiddelde_uit_scoort(self):
        sum_aas = 0
        for team in self.teams:
            sum_aas += team.aas
        
        return sum_aas/len(self.teams)
    
    def gemiddelde_uit_toegegeven(self):
        sum_aac = 0
        for team in self.teams:
            sum_aac += team.aac
        
        return sum_aac/len(self.teams)
    
    def thuis_aanvalskracht(self, thuisploeg): # 'Attack Strength'
            return thuisploeg.ahs/ self.gemiddelde_thuis_scoort()
    
    def uit_aanvalskracht(self, tegenstander): # 'Attack Strength'
            return tegenstander.aas/ self.gemiddelde_uit_scoort()
    
    def thuis_verdedigingskracht(self, thuisploeg): # 'Defense Strength'
            return thuisploeg.ahc/ self.gemiddelde_thuis_toegegeven()
    
    def uit_verdedigingskracht(self, tegenstander): # 'Defense Strength'
            return tegenstander.aac/ self.gemiddelde_uit_toegegeven()
    
    def thuis_doelpunten(self, thuisploeg, tegenstander):
        return self.thuis_aanvalskracht(thuisploeg) * self.uit_verdedigingskracht(tegenstander) * self.gemiddelde_thuis_scoort()
    
    def uit_doelpunten(self, thuisploeg, tegenstander):
        return self.uit_aanvalskracht(thuisploeg) * self.thuis_verdedigingskracht(tegenstander) * self.gemiddelde_uit_scoort()
        
    def poisson_verdeling(self, expected_occurences, discrete_events):
        # https://towardsdatascience.com/the-poisson-distribution-and-poisson-process-explained-4e2cb17d459
        #TODO verandert var namen naar NL 
        return math.e**(-expected_occurences) * ((expected_occurences**discrete_events)/math.factorial(discrete_events))
    
    
    def match_outcome(self, rng, thuisploeg, tegenstander):
        kans_doelpunten_verdeling = 0
        r = 10 # can passed in the function , maybe
        for x in range(r+1):
            # kiko ta e chens between 0-0, 0-1, 1-0, 1-1
            for y in range(r+1):
                thuis_kans_doelpunt = self.poisson_verdeling( self.thuis_doelpunten(thuisploeg, tegenstander), x)
                uit_kans_doelpunt   = self.poisson_verdeling( self.uit_doelpunten(thuisploeg, tegenstander), y)
                
                kans_doelpunten_verdeling += thuis_kans_doelpunt * uit_kans_doelpunt
                doelpunten_verdeling = "{} - {}".format(x, y)
                
                if rng <= kans_doelpunten_verdeling:
                    return x, y
                
                
        
                
        
        
        

In [ ]:
t = Tournament()
ajax = Team(3.2, 0.9, 3.1, 0.6)
feyenoord = Team(2.4, 1.1, 2.2, 0.8)
t.teams.append(ajax)
t.teams.append(feyenoord)

In [ ]:
t.poisson_verdeling(1.623, 5) *100

In [ ]:
t.match_outcome(random.random(), ajax, feyenoord )